# Générer du texte avec des RNN

Nous utiliserons pour cela un module adapté par Donald Dong qui permet d'entraîner rapidement un RNN pour la génération de texte, appelé `rnn-text-gen`. La documentation autour du module en question est [disponible sur Github](https://github.com/donaldong/rnn-text-gen).

Contrairement au module `markovify`, nous n'allons pas installer le module et l'importer comme n'importe quel module Python. Nous allons ici devoir récupérer le répertoire git et l'importer ici.

On commence pour cela par cloner le répertoire Git.

In [1]:
!git clone https://github.com/donaldong/rnn-text-gen.git

致命错误：目标路径 'rnn-text-gen' 已经存在，并且不是一个空目录。


Pensez à remplacer le fichier 'text_generator.py' présent dans le répertoire 'rnn-text-gen/src' par le fichier disponible sur iCampus afin de pouvoir faire tourner le reste du code sous Tensorflow 2. Voir les détails de la manipulation sur iCampus.

Il nous faut alors dire au système (ici, notre système virtuel) où se trouve le module que l'on a récupé. On le fait à l'aide du code dans la cellule suivante.

In [2]:
import sys
sys.path.insert(0,'/content/rnn-text-gen')



Il nous faut maintenant importer les classes et libraires pertinentes

In [3]:
from src.text_generator import RNNTextGenerator
from src.dataset import Dataset
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2023-01-27 00:42:30.173424: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: module 'tensorflow._api.v2.nn' has no attribute 'rnn_cell'

*On* peut alors commencer à entraîner notre RNN.

On commence par charger notre corpus d'entraînement. Ici, le module vient avec quelques corpus, dont on peut regarder rapidement à quoi ils ressemblent.

In [9]:
f = open('data/cuisine.txt')
text = f.read()
print(text[:1000])

Blanquette de veau
{{Voir homonymes|Blanquette|Veau (homonymie)}}
{{Infobox Mets
 | nom            = Blanquette de veau
 | image          = Blanquette de veau à l'ancienne 04.jpg
 | légende        = Blanquette de veau à l'ancienne et riz.
 | autre nom      = 
 | lieu origine   = {{France}} 
 | créateur       = 
 | date           = Ancienne
 | place service  = [[Plat principal]]
 | température    = Chaude
 | ingrédients    = [[Viande]] de [[veau]], [[carotte]], [[poireau]], [[oignon grelot]], [[champignon de Paris]], [[Liaison (cuisine)|liée]] en [[sauce blanche]] à la [[crème fraîche|crème]] et au [[beurre]]
 | variations     = Cuisine à la [[Crème (produit laitier)|crème]], [[ragoût]], [[bœuf Stroganov]], [[poularde aux morilles]], [[daube provençale]], [[miroton de bœuf]]
 | accompagnement = [[Riz]], [[Pâtes alimentaires|pâtes]], [[pomme de terre|pommes de terre]], [[vin]] du [[Viticulture en France|vignoble français]] 
 | classification = [[Cuisine française]]
}}
La '''blanquette'''

Il nous faut maintenant définir les paramètres d'entraînement. Parmi ces paramètres, on trouve :
- La taille des batchs (batch_size)
- La longueur des séquences (seq_length)
- Le nombre d'époch (epoch)
- Le taux d'apprentissage (learning_rate)

In [10]:
seq_length = 25
learning_rate = 0.02
epoch = 1
batch_size = 40

On peut alors traiter le corpus, c'est-à-dire créer les vecteurs qui serviront d'input au RNN. Le présent module propose par la fonction Dataset de transformer le corpus en un ensemble de batchs comprenant *n* séquences de longueur *j* de vecteurs one-hot (où *n* correspond à la valeur définie par batch_size, et *j* à la longueur des séquences définie par seq_length)

Notons qu'ici, la fonction crée des vecteurs pour les caractères, mais on peut de façon générale choisir de faire des vecteurs pour les mots eux-mêmes et non les caractères.

De même, les RNNs peuvent prendre en entrée des vecteurs de type *word embeddings*, mais nous nous limiterons ici aux vecteurs one-hot.


In [11]:
alice_dataset = Dataset(['data/cuisine.txt'], seq_length)
print (alice_dataset)

On peut alors entraîner le modèle à l'aide de la fonction `RNNTextGenerator()`. Un argument important de cette fonction est le type d'unités que l'on utilise (ici BasicRNNCell). On pourra plus tard utiliser des unités de type LSTM ou GRU pour entraîner les réseaux de neurones correspondants.

In [12]:
rnn_model = RNNTextGenerator(
    seq_length,
    alice_dataset.vocab_size,
    rnn_cell=tf.compat.v1.nn.rnn_cell.LSTMCell,
    learning_rate=learning_rate,
    epoch=epoch,
    batch_size=batch_size,
)




Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2023-01-27 00:29:00.705710: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 00:29:00.706511: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Instructions for updating:
Use keras.layers.dense instead.





Une façon d'évaluer nos paramètres et le degré de convergence de notre RNN consiste à voir comment évoluent la précision (Accuracy) et la fonction objectif (loss function).

On commence par faire converger le modèle sur nos données, et on sauvegarde les scores dans une variable.

In [13]:
rnn_model_scores = rnn_model.fit(
    alice_dataset,
    save_scores=True
)

KeyboardInterrupt: 

On peut alors produire une visualisation graphique de l'évolution de ces scores.

In [ ]:
fig, axes = plt.subplots(figsize=(15, 6), ncols=2)
rnn_model_scores['accuracy'].plot(
    ax=axes[0], title='Accuracy (Train)'
)
rnn_model_scores['loss'].plot(
    ax=axes[1], title='Loss (Train)'
)
for ax in axes:
    ax.set(xlabel='Steps')

La courbe de la fonction objectif doit normalement une courbe descendante d'abord rapide puis qui ralentit. Si la courbe ne se stabilise cependant pas (qu'il y a beaucoup de pics par exemple), on considère que le RNN n'a pas convergé. Un modèle ne converge pas quand la fonction objectif ne se stabilise pas. Une raison à cette non-convergence peut s'expliquer par le nombre limité d'epochs (1) que nous avons fixé. On peut donc choisir de réentraîner notre RNN avec un nombre d'epochs plus élevé, par exemple.

Sachez que l'on peut aussi affiner notre modèle sans le réentraîner de 0 en le va le 'fit' sur nos données (on peut considérer cela comme un epoch). On peut ainsi demander au modèle de s'affiner un certain nombre de fois, ou durant une certaine période, comme on le fait ci-dessous.

De façon assez intéressante, on peut suivre l'évolution en direct de ces scores et de l'état de la prédiction en temps réel à mesure que le RNN s'entraîne.

In [52]:
# On importe la fonction native du module permettant de fixer une durée maximale à l'entraînement
from src.time_limit import time_limit

# On définit la durée de l'entraînement
for _ in time_limit(seconds=30): 
  # On affine le modèle
  rnn_model.fit(alice_dataset)
  acc, loss = rnn_model.score(alice_dataset)
  # On imprime à chaque epoch la précision et la valeur de la loss function
  print('test acc: {}, test loss: {}'.format(
        acc, loss
    ))
  # On demande à chaque epoch d'imprimer une séquence de 100 caractères prédits à partir de la séquence initiale 'Yes, but '
  start_seq = 'Yes, but '
  print(start_seq + rnn_model.generate(
        alice_dataset,
        start_seq,
        100
    ))
  print('-----------------------')

test acc: 0.579800009727478, test loss: 1.4071494340896606


/content/rnn-text-gen/src/text_generator.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  self._tf_rnn_cell = rnn_cell(
/content/rnn-text-gen/src/text_generator.py:115: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  logits = tf.compat.v1.layers.dense(


Yes, but the
Knave beck cealded the Mouse?' the March Hare.

Alice looked upon and was hand, without was tif-
-----------------------


Quelque soit la performance de votre RNN, il est en tout cas déjà prêt pour la génération de phrase. Générer une séquence se fait à l'aide de la fonction `generate()`. La fonction prend en argument le dataset (c'est-à-dire le dictionnaire contenant les équivalences entre les vecteurs one-hot et les symboles correspondants), la séquence à partir de laquelle prédire, et le nombre de caractères à prédire.

In [53]:
# Pour des raisons de lisibilité, on définit la séquence d'amorce dans une variable à part
start_seq = 'Of course '

# On imprime ici directement l'amorce et le texte généré (mais on pourrait enregistrer le texte généré dans une variable par exemple)
print(start_seq + rnn_model.generate(
        alice_dataset,
        start_seq,
        500
    ))

Of course beht quite as
she only deares as preaced himely.

'I'm
and not, me almos. The natched intind about the Sea-bless?' said the Mouse, what usuoped optor,' the Dodo,
way just
all fir at the Knall reme!'
I want to the went on this: was it my way with their to:
'I never coppreach criam there is a routs he
dreadly way?' said the Hattier, croseding man, that and
fiued the creacle peglind deal something on the
jury did talking at the Dodow midd silbing in tit!' Ger execute beht some say that sooting hand


Comme dit plus haut, vous pouvez entraîner des RNN plus spécifiques, à savoir des LSTM et des réseaux de type GRU. Il suffit pour cela notamment de remplacer dans la fonction d'entraînement le `BasicRNNCell` par `LSTMCell` ou `GRUCell` (et modifier les valeurs des autres arguments si besoin).

## Exercice

- Quels sont les paramètres qui vous permettent d'améliorer les résultats pour un petit corpus ?
- Quel est l'impact de la taille du corpus ?
- Quelle est la performance du RNN sur la prédiction du wikiCréole ?
- Quel est le type de réseau de neurones qui marche le mieux (sur vos données) ?